在自然语言处理任务中，词向量（Word Embedding）是表示自然语言里单词的一种方法，即把每个词都表示为一个N维空间内的点，即一个高维空间内的向量。通过这种方法，实现把自然语言计算转换为向量计算。

如 **图1** 所示的词向量计算任务中，先把每个词（如queen，king等）转换成一个高维空间的向量，这些向量在一定意义上可以代表这个词的语义信息。再通过计算这些向量之间的距离，就可以计算出词语之间的关联关系，从而达到让计算机像计算数值一样去计算自然语言的目的。

<center><img src="https://ai-studio-static-online.cdn.bcebos.com/00ba55f7304e4f97942165cf1deb946ced404a19325d40969b7c220e30cf527e" width="800" ></center>
<center>图1：词向量计算示意图</center>
<br></br>

因此，大部分词向量模型都需要回答两个问题：

1. **如何把词转换为向量?**

自然语言单词是离散信号，比如“香蕉”，“橘子”，“水果”在我们看来就是3个离散的词。

如何把每个离散的单词转换为一个向量？

2. **如何让向量具有语义信息?**

比如，我们知道在很多情况下，“香蕉”和“橘子”更加相似，而“香蕉”和“橘子”就没有那么相似，同时“香蕉”和“食物”、“水果”的相似程度可能介于“橘子”和“句子”之间。

那么，我们该如何让词向量具备这样的语义信息？


# 如何把词转换为向量

自然语言单词是离散信号，比如“我”、“ 爱”、“人工智能”。如何把每个离散的单词转换为一个向量？通常情况下，我们可以维护一个如 **图2** 所示的查询表。表中每一行都存储了一个特定词语的向量值，每一列的第一个元素都代表着这个词本身，以便于我们进行词和向量的映射（如“我”对应的向量值为 [0.3，0.5，0.7，0.9，-0.2，0.03] ）。给定任何一个或者一组单词，我们都可以通过查询这个excel，实现把单词转换为向量的目的，这个查询和替换过程称之为Embedding Lookup。
<center><img src="https://ai-studio-static-online.cdn.bcebos.com/2ccf57a0c4584ca5b5000da85bc48c4cbe1588f0e3f548c6b2271c26e2e7df14" width="800" ></center>
<center><br>图2：词向量查询表</br></center>
<br></br>

上述过程也可以使用一个字典数据结构实现。事实上如果不考虑计算效率，使用字典实现上述功能是个不错的选择。然而在进行神经网络计算的过程中，需要大量的算力，常常要借助特定硬件（如GPU）满足训练速度的需求。GPU上所支持的计算都是以张量（Tensor）为单位展开的，因此在实际场景中，我们需要把Embedding Lookup的过程转换为张量计算，如 **图3** 所示。
<center><img src="https://ai-studio-static-online.cdn.bcebos.com/0da8ca9f87364701a95a5d1f51736dd08bd1e6321a3f4c30bfb2ee2d2385698c" width="800" ></center>
<center><br>图3：张量计算示意图</br></center>
<br></br>

假设对于句子"我，爱，人工，智能"，把Embedding Lookup的过程转换为张量计算的流程如下：

1. 通过查询字典，先把句子中的单词转换成一个ID（通常是一个大于等于0的整数），这个单词到ID的映射关系可以根据需求自定义（如**图3**中，我=>1, 人工=>2，爱=>3，...）。

2. 得到ID后，再把每个ID转换成一个固定长度的向量。假设字典的词表中有5000个词，那么，对于单词“我”，就可以用一个5000维的向量来表示。由于“我”的ID是1，因此这个向量的第一个元素是1，其他元素都是0（[1，0，0，…，0]）；同样对于单词“人工”，第二个元素是1，其他元素都是0。用这种方式就实现了用一个向量表示一个单词。由于每个单词的向量表示都只有一个元素为1，而其他元素为0，因此我们称上述过程为One-Hot Encoding。

3. 经过One-Hot Encoding后，句子“我，爱，人工，智能”就被转换成为了一个形状为 4×5000的张量，记为$V$。在这个张量里共有4行、5000列，从上到下，每一行分别代表了“我”、“爱”、“人工”、“智能”四个单词的One-Hot Encoding。最后，我们把这个张量$V$和另外一个稠密张量$W$相乘，其中$W$张量的形状为5000 × 128（5000表示词表大小，128表示每个词的向量大小）。经过张量乘法，我们就得到了一个4×128的张量，从而完成了把单词表示成向量的目的。

# 如何让向量具有语义信息

得到每个单词的向量表示后，我们需要思考下一个问题：比如在多数情况下，“香蕉”和“橘子”更加相似，而“香蕉”和“句子”就没有那么相似；同时，“香蕉”和“食物”、“水果”的相似程度可能介于“橘子”和“句子”之间。那么如何让存储的词向量具备这样的语义信息呢？

我们先学习自然语言处理领域的一个小技巧。在自然语言处理研究中，科研人员通常有一个共识：使用一个单词的上下文来了解这个单词的语义，比如：

 >“苹果手机质量不错，就是价格有点贵。”
 >
 >“这个苹果很好吃，非常脆。”
 >
 >“菠萝质量也还行，但是不如苹果支持的APP多。”
>
在上面的句子中，我们通过上下文可以推断出第一个“苹果”指的是苹果手机，第二个“苹果”指的是水果苹果，而第三个“菠萝”指的应该也是一个手机。事实上，在自然语言处理领域，使用上下文描述一个词语或者元素的语义是一个常见且有效的做法。我们可以使用同样的方式训练词向量，让这些词向量具备表示语义信息的能力。

2013年，Mikolov提出的经典word2vec算法就是通过上下文来学习语义信息。word2vec包含两个经典模型：CBOW（Continuous Bag-of-Words）和Skip-gram，如 **图4** 所示。

- **CBOW**：通过上下文的词向量推理中心词。
- **Skip-gram**：根据中心词推理上下文。

<center><img src="https://ai-studio-static-online.cdn.bcebos.com/87b90136eef04d7285803e567a5f7f6d0e40bb552deb40a19a7540c4e6aa20a3" width="700" ></center>
<center><br>图4：CBOW和Skip-gram语义学习示意图</br></center>
<br></br>

假设有一个句子“Pineapples are spiked and yellow”，两个模型的推理方式如下：

- 在**CBOW**中，先在句子中选定一个中心词，并把其它词作为这个中心词的上下文。如 **图4** CBOW所示，把“Spiked”作为中心词，把“Pineapples、are、and、yellow”作为中心词的上下文。在学习过程中，使用上下文的词向量推理中心词，这样中心词的语义就被传递到上下文的词向量中，如“Spiked → pineapple”，从而达到学习语义信息的目的。

- 在**Skip-gram**中，同样先选定一个中心词，并把其他词作为这个中心词的上下文。如 **图4** Skip-gram所示，把“Spiked”作为中心词，把“Pineapples、are、and、yellow”作为中心词的上下文。不同的是，在学习过程中，使用中心词的词向量去推理上下文，这样上下文定义的语义被传入中心词的表示中，如“pineapple → Spiked”，
从而达到学习语义信息的目的。

------
**说明：**

一般来说，CBOW比Skip-gram训练速度快，训练过程更加稳定，原因是CBOW使用上下文average的方式进行训练，每个训练step会见到更多样本。而在生僻字（出现频率低的字）处理上，skip-gram比CBOW效果更好，原因是skip-gram不会刻意回避生僻字(CBOW结构中输入中存在生僻字时，生僻字会被其它非生僻字的权重冲淡)。

------

## CBOW和Skip-gram的算法实现

我们以这句话：“Pineapples are spiked and yellow”为例分别介绍CBOW和Skip-gram的算法实现。

如 **图5** 所示，CBOW是一个具有3层结构的神经网络，分别是：

<center><img src="https://ai-studio-static-online.cdn.bcebos.com/72397490c0ba499692cff31484431c57bc9d20f7ef344454868e12d628ec5bd3" width="400" ></center>
<center><br>图5：CBOW的算法实现</br></center>
<br></br>

* **输入层：** 一个形状为C×V的one-hot张量，其中C代表上线文中词的个数，通常是一个偶数，我们假设为4；V表示词表大小，我们假设为5000，该张量的每一行都是一个上下文词的one-hot向量表示，比如“Pineapples, are, and, yellow”。
* **隐藏层：** 一个形状为V×N的参数张量W1，一般称为word-embedding，N表示每个词的词向量长度，我们假设为128。输入张量和word embedding W1进行矩阵乘法，就会得到一个形状为C×N的张量。综合考虑上下文中所有词的信息去推理中心词，因此将上下文中C个词相加得一个1×N的向量，是整个上下文的一个隐含表示。
* **输出层：** 创建另一个形状为N×V的参数张量，将隐藏层得到的1×N的向量乘以该N×V的参数张量，得到了一个形状为1×V的向量。最终，1×V的向量代表了使用上下文去推理中心词，每个候选词的打分，再经过softmax函数的归一化，即得到了对中心词的推理概率：

$$𝑠𝑜𝑓𝑡𝑚𝑎𝑥({O_i})= \frac{exp({O_i})}{\sum_jexp({O_j})}$$

如 **图6** 所示，Skip-gram是一个具有3层结构的神经网络，分别是：

<center><img src="https://ai-studio-static-online.cdn.bcebos.com/a572953b845d4c91bdf6b7b475e7b4437bee69bd60024eb2b8c46f56adf2bdef" width="400" ></center>
<center><br>图6：Skip-gram算法实现</br></center>
<br></br>

- **Input Layer（输入层）**：接收一个one-hot张量 $V \in R^{1 \times \text{vocab\_size}}$ 作为网络的输入，里面存储着当前句子中心词的one-hot表示。
- **Hidden Layer（隐藏层）**：将张量$V$乘以一个word embedding张量$W_1 \in R^{\text{vocab\_size} \times \text{embed\_size}}$，并把结果作为隐藏层的输出，得到一个形状为$R^{1 \times \text{embed\_size}}$的张量，里面存储着当前句子中心词的词向量。
- **Output Layer（输出层）**：将隐藏层的结果乘以另一个word embedding张量$W_2 \in R^{\text{embed\_size} \times \text{vocab\_size}}$，得到一个形状为$R^{1 \times \text{vocab\_size}}$的张量。这个张量经过softmax变换后，就得到了使用当前中心词对上下文的预测结果。根据这个softmax的结果，我们就可以去训练词向量模型。

在实际操作中，使用一个滑动窗口（一般情况下，长度是奇数），从左到右开始扫描当前句子。每个扫描出来的片段被当成一个小句子，每个小句子中间的词被认为是中心词，其余的词被认为是这个中心词的上下文。

   
### Skip-gram的理想实现

使用神经网络实现Skip-gram中，模型接收的输入应该有2个不同的tensor：

- 代表中心词的tensor：假设我们称之为center_words $V$，一般来说，这个tensor是一个形状为[batch_size, vocab_size]的one-hot tensor，表示在一个mini-batch中，每个中心词的ID，对应位置为1，其余为0。

- 代表目标词的tensor：目标词是指需要推理出来的上下文词，假设我们称之为target_words $T$，一般来说，这个tensor是一个形状为[batch_size, 1]的整型tensor，这个tensor中的每个元素是一个[0, vocab_size-1]的值，代表目标词的ID。

在理想情况下，我们可以使用一个简单的方式实现skip-gram。即把需要推理的每个目标词都当成一个标签，把skip-gram当成一个大规模分类任务进行网络构建，过程如下：

1. 声明一个形状为[vocab_size, embedding_size]的张量，作为需要学习的词向量，记为$W_0$。对于给定的输入$V$，使用向量乘法，将$V$乘以$W_0$，这样就得到了一个形状为[batch_size, embedding_size]的张量，记为$H=V×W_0$。这个张量$H$就可以看成是经过词向量查表后的结果。
1. 声明另外一个需要学习的参数$W_1$，这个参数的形状为[embedding_size, vocab_size]。将上一步得到的$H$去乘以$W_1$，得到一个新的tensor $O=H×W_1$，此时的$O$是一个形状为[batch_size, vocab_size]的tensor，表示当前这个mini-batch中的每个中心词预测出的目标词的概率。
1. 使用softmax函数对mini-batch中每个中心词的预测结果做归一化，即可完成网络构建。

   
### Skip-gram的实际实现

然而在实际情况中，vocab_size通常很大（几十万甚至几百万），导致$W_0$和$W_1$也会非常大。对于$W_0$而言，所参与的矩阵运算并不是通过一个矩阵乘法实现，而是通过指定ID，对参数$W_0$进行访存的方式获取。然而对$W_1$而言，仍要处理一个非常大的矩阵运算（计算过程非常缓慢，需要消耗大量的内存/显存）。为了缓解这个问题，通常采取负采样（negative_sampling）的方式来近似模拟多分类任务。此时新定义的$W_0$和$W_1$均为形状为[vocab_size, embedding_size]的张量。

假设有一个中心词$c$和一个上下文词正样本$t_p$。在Skip-gram的理想实现里，需要最大化使用$c$推理$t_p$的概率。在使用softmax学习时，需要最大化$t_p$的推理概率，同时最小化其他词表中词的推理概率。之所以计算缓慢，是因为需要对词表中的所有词都计算一遍。然而我们还可以使用另一种方法，就是随机从词表中选择几个代表词，通过最小化这几个代表词的概率，去近似最小化整体的预测概率。比如，先指定一个中心词（如“人工”）和一个目标词正样本（如“智能”），再随机在词表中采样几个目标词负样本（如“日本”，“喝茶”等）。有了这些内容，我们的skip-gram模型就变成了一个二分类任务。对于目标词正样本，我们需要最大化它的预测概率；对于目标词负样本，我们需要最小化它的预测概率。通过这种方式，我们就可以完成计算加速。上述做法，我们称之为负采样。

在实现的过程中，通常会让模型接收3个tensor输入：

- 代表中心词的tensor：假设我们称之为center_words $V$，一般来说，这个tensor是一个形状为[batch_size, vocab_size]的one-hot tensor，表示在一个mini-batch中每个中心词具体的ID。

- 代表目标词的tensor：假设我们称之为target_words $T$，一般来说，这个tensor同样是一个形状为[batch_size, vocab_size]的one-hot tensor，表示在一个mini-batch中每个目标词具体的ID。

- 代表目标词标签的tensor：假设我们称之为labels $L$，一般来说，这个tensor是一个形状为[batch_size, 1]的tensor，每个元素不是0就是1（0：负样本，1：正样本）。

模型训练过程如下：
1. 用$V$去查询$W_0$，用$T$去查询$W_1$，分别得到两个形状为[batch_size, embedding_size]的tensor，记为$H_1$和$H_2$。
1. 点乘这两个tensor，最终得到一个形状为[batch_size]的tensor  $O = [O_i = \sum_j H_0[i,j] × H_1[i,j]]_{i=1}^{batch\_size}$。
1. 使用sigmoid函数作用在$O$上，将上述点乘的结果归一化为一个0-1的概率值，作为预测概率，根据标签信息$L$训练这个模型即可。

在结束模型训练之后，一般使用$W_0$作为最终要使用的词向量，可以用$W_0$提供的向量表示。通过向量点乘的方式，计算两个不同词之间的相似度。

